In [1]:
!pip install koeda

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 37.1 MB/s eta 0:00:00
  Attempting uninstall: tweepy
    Found existing installation: tweepy 4.15.0
    Uninstalling tweepy-4.15.0:
      Successfully uninstalled tweepy-4.15.0


In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from transformers import TFBertModel, BertConfig, AutoTokenizer, TFBertForSequenceClassification, TFAutoModel
from sklearn.model_selection import train_test_split
from tensorflow import keras
import tf_keras
from tf_keras.src.layers import Input, Dense, Dropout
from tf_keras.src.models import Model

In [3]:
from tf_keras.metrics import Metric
from tf_keras.metrics import Precision, Recall

class MacroF1Score(Metric):
    def __init__(self, num_classes, name='macro_f1_score', **kwargs):
        super(MacroF1Score, self).__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.precision = [tf.keras.metrics.Precision(class_id=i) for i in range(num_classes)]
        self.recall = [tf.keras.metrics.Recall(class_id=i) for i in range(num_classes)]

    def update_state(self, y_true, y_pred, sample_weight=None):
        # y_pred: 소프트맥스 출력, y_true: 정수 라벨
        y_pred = tf.argmax(y_pred, axis=-1)  # 예측 클래스
        y_true = tf.cast(y_true, tf.int64)   # 정수형 라벨
        for i in range(self.num_classes):
            y_true_i = tf.cast(tf.equal(y_true, i), tf.float32)
            y_pred_i = tf.cast(tf.equal(y_pred, i), tf.float32)
            self.precision[i].update_state(y_true_i, y_pred_i, sample_weight)
            self.recall[i].update_state(y_true_i, y_pred_i, sample_weight)

    def result(self):
        f1_scores = []
        for i in range(self.num_classes):
            p = self.precision[i].result()
            r = self.recall[i].result()
            f1 = 2 * (p * r) / (p + r + tf.keras.backend.epsilon())
            f1_scores.append(f1)
        return tf.reduce_mean(f1_scores)  # 매크로 평균

    def reset_states(self):
        for i in range(self.num_classes):
            self.precision[i].reset_states()
            self.recall[i].reset_states()

In [4]:
def get_token_data(data, tokenizer):
    return_data = tokenizer(
    data,
    padding='max_length', # 자동으로 최대 길이로 패딩해줌
    truncation=True, # 모델이 감당 가능한 최대 길이 초과하면 자름
    return_tensors='np',
    # max_length = token_max_len)
    )
    return return_data

In [5]:
print(tf.__version__)

2.18.0


In [6]:
# 사용 가능한 GPU 목록 확인
gpus = tf.config.list_physical_devices('GPU')
print("Num GPUs Available:", len(gpus))
print("GPU Details:", gpus)

Num GPUs Available: 1
GPU Details: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### 데이터 전처리

In [7]:
class_dict = {'협박 대화': 0, '갈취 대화':1, '직장 내 괴롭힘 대화':2, '기타 괴롭힘 대화':3, '일반 대화':4}

In [8]:
train_df = pd.read_csv('trainset_v1.csv') # 캐글 데이터
val_df = pd.read_csv('valset_v1.csv') # 합성데이터

In [9]:
train_df['conversation'] = train_df['conversation'].str.replace('\n', ' ')
val_df['conversation'] = val_df['conversation'].str.replace('\n', ' ')

In [10]:
all_df = pd.concat([train_df, val_df])

In [11]:
# 모든 데이터의 공백 제거 최대 길이를 구함
data_len = [len(x.split()) for x in all_df['conversation']]
MAX_LEN = max(data_len)
MAX_LEN

223

In [12]:
# 훈련 데이터 class 개수
train_df['class'].value_counts()

,count
class,
3,809
4,798
1,778
2,776
0,714


In [13]:
# 훈련 데이터 class 개수
val_df['class'].value_counts()

,count
class,
3,202
4,200
1,195
2,194
0,178


In [14]:
num_classes = len(np.unique(train_df['class']))
print(f'class 개수 : {num_classes}')

class 개수 : 5


In [15]:
from koeda import AEDA
from IPython.display import clear_output
import time

aeda = AEDA(
    morpheme_analyzer="Okt", punc_ratio=0.1
)

aug_class = []
aug_data = []

def safe_aeda(aeda, text):
    try:
        return aeda(text)
    except Exception as e:
        print(f"[AEDA ERROR] 텍스트: '{text}' → {e}")
        return text

for i, d in train_df.iterrows():
    clear_output(wait=True)
    print(f"[{i}] 증강 중...")
    print(d["class"])
    aug_class.append(d["class"])
    #aug_data.append(aeda(d["conversation"]))
    aug_data.append(safe_aeda(aeda, d["conversation"]))

[3874] 증강 중...
4


In [16]:
aug_df = pd.DataFrame({
    "class": aug_class,
    "conversation": aug_data
})

aug_df.head()

,class,conversation
0,0,"너네 딸 어디있어? 누구 . 시죠? 너 , 네 딸이 우리 딸 이렇게 만들었다고! 당..."
1,0,! 입금받았으니 물건 빨리 보내시죠 당분간 바빠서 그래요 장난하십니까.벌써 몇번째에...
2,2,"이사원 이 , 업무좀 내일까지 할수있나 저 지금 하고있는거도있고 내일 ! 까지는 힘..."
3,1,안녕 꼬마야 몇살이야 안녕하세요 6살이요 손에 든건뭐야? 돈이에요 엄마가 애호박 ....
4,1,아니 이자가 왜 이리 늘어요? 너무한거 아니에요 뭐 새끼야 애초에 돈을 빌렸으면 돈...


In [17]:
train_df.head()

,class,conversation
0,0,너네 딸 어디있어? 누구시죠? 너네 딸이 우리 딸 이렇게 만들었다고! 당장 데려...
1,0,입금받았으니 물건 빨리 보내시죠 당분간 바빠서 그래요 장난하십니까.벌써 몇번째에...
2,2,이사원 이업무좀 내일까지 할수있나 저 지금 하고있는거도있고 내일까지는 힘들거같습니다...
3,1,안녕 꼬마야 몇살이야 안녕하세요 6살이요 손에 든건뭐야? 돈이에요 엄마가 애호박 하...
4,1,아니 이자가 왜 이리 늘어요? 너무한거 아니에요 뭐 새끼야 애초에 돈을 빌렸으면 돈...


In [18]:
train_df = pd.concat([train_df, aug_df], ignore_index=True)

In [19]:
aug_df.to_csv('baseline_pretrained_rm_729_aeda_aug.csv')

### 토크나이저

In [20]:
model_name = "klue/bert-base"

In [21]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [22]:
train_labels = tf.convert_to_tensor(train_df['class'])
train_token_data = get_token_data(list(train_df['conversation']), tokenizer)
train_token_data = {key: tf.convert_to_tensor(train) for key, train in train_token_data.items()}

In [23]:
val_labels = tf.convert_to_tensor(val_df['class'])
val_token_data = get_token_data(list(val_df['conversation']), tokenizer)
val_token_data = {key: tf.convert_to_tensor(val) for key, val in val_token_data.items()}

In [24]:
train_token_data

{'input_ids': <tf.Tensor: shape=(7750, 512), dtype=int64, numpy=
 array([[    2,   743,  2203, ...,     0,     0,     0],
        [    2, 12793,  2757, ...,     0,     0,     0],
        [    2,  4188,  2252, ...,     0,     0,     0],
        ...,
        [    2,   732,  2116, ...,     0,     0,     0],
        [    2, 26784,  2052, ...,     0,     0,     0],
        [    2,  1453,    35, ...,     0,     0,     0]])>,
 'token_type_ids': <tf.Tensor: shape=(7750, 512), dtype=int64, numpy=
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])>,
 'attention_mask': <tf.Tensor: shape=(7750, 512), dtype=int64, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]])>}

In [25]:
train_labels

<tf.Tensor: shape=(7750,), dtype=int64, numpy=array([0, 0, 2, ..., 1, 3, 4])>

In [26]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_token_data, train_labels))
train_dataset = train_dataset.shuffle(buffer_size=10000).batch(8) # 셔플 및 배치

# 예시: 검증 데이터셋 생성
val_dataset = tf.data.Dataset.from_tensor_slices((val_token_data, val_labels))
val_dataset = val_dataset.batch(8) # 검증 데이터는 보통 셔플하지 않음

In [27]:
lengths = [len(seq) for seq in train_token_data['input_ids']]
print(f"토크나이저 후 데이터 내 최대 시퀀스 길이: {max(lengths)}")

토크나이저 후 데이터 내 최대 시퀀스 길이: 512


In [28]:
for i in train_dataset:
    print(i)
    break

({'input_ids': <tf.Tensor: shape=(8, 512), dtype=int64, numpy=
array([[   2, 7772,    5, ...,    0,    0,    0],
       [   2,  647, 2328, ...,    0,    0,    0],
       [   2, 1396,    5, ...,    0,    0,    0],
       ...,
       [   2, 3767, 2454, ...,    0,    0,    0],
       [   2,  766, 2104, ...,    0,    0,    0],
       [   2, 4100, 9769, ...,    0,    0,    0]])>, 'token_type_ids': <tf.Tensor: shape=(8, 512), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>, 'attention_mask': <tf.Tensor: shape=(8, 512), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>}, <tf.Tensor: shape=(8,), dtype=int64, numpy=array([3, 2, 3, 1, 3, 3, 2, 3])>

### 모델

In [29]:
# 사전학습된 klue 모델의 bert 인코더만 불러오기 -> TFAutoModel 사용

bert_encoders = TFAutoModel.from_pretrained(model_name, num_labels=num_classes, from_pt=True)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'bert.embeddings.position_ids', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [30]:
# 4. 더미 입력으로 모델 빌드
dummy_input = get_token_data(["더미 텍스트"], tokenizer)
# bert_encoders(dummy_input)  # 모델 호출로 빌드

In [31]:
input_layers = {}
for name, tensor in dummy_input.items(): # 토크나이저의 출력값을 알기 위해서 더미데이터 출력값 가져옴
    shape = tensor.shape[1:]  # batch dimension 제외
    input_layers[name] = Input(shape=shape, dtype='int32', name=name) # input_layer 정의 ( tokneizer의 출력값 전부가 input으로 들어감 )
input_layers

{'input_ids': <KerasTensor: shape=(None, 512) dtype=int32 (created by layer 'input_ids')>,
 'token_type_ids': <KerasTensor: shape=(None, 512) dtype=int32 (created by layer 'token_type_ids')>,
 'attention_mask': <KerasTensor: shape=(None, 512) dtype=int32 (created by layer 'attention_mask')>}

In [32]:
# bert 인코더 통과 시키고
bert_outputs = bert_encoders(input_layers)
pooled_output = bert_outputs.pooler_output

In [33]:
# dense 층 추가
dense1 = Dense(1024, activation='relu')(pooled_output)
dropout1 = Dropout(0.5)(dense1)
dense2 = Dense(512, activation='relu')(dropout1)
dropout2 = Dropout(0.5)(dense2)
classifier1 = Dense(num_classes)(dropout2)

In [34]:
model = Model(inputs=input_layers, outputs=classifier1)

In [35]:
# 사전학습된 bert 인코더 훈련 X

bert_encoders.trainable = False

In [36]:
for layer in model.layers:
    print(f"Layer {layer.name} is trainable: {layer.trainable}")

Layer attention_mask is trainable: True
Layer input_ids is trainable: True
Layer token_type_ids is trainable: True
Layer tf_bert_model is trainable: False
Layer dense is trainable: True
Layer dropout_37 is trainable: True
Layer dense_1 is trainable: True
Layer dropout_38 is trainable: True
Layer dense_2 is trainable: True


In [37]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 attention_mask (InputLayer  [(None, 512)]                0         []                            
 )                                                                                                
                                                                                                  
 input_ids (InputLayer)      [(None, 512)]                0         []                            
                                                                                                  
 token_type_ids (InputLayer  [(None, 512)]                0         []                            
 )                                                                                                
                                                                                              

In [38]:
from tf_keras.optimizers import Adam  # tf_keras에서 Adam 가져오기
f1 = MacroF1Score(num_classes=num_classes)
optimizer = Adam(learning_rate=5e-5)
loss = tf_keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy', f1])
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [39]:
early_stopping_cb = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    restore_best_weights=True,
    patience=5)

# # ModelCheckpoint 콜백 수정
# model_checkpoint_cb = keras.callbacks.ModelCheckpoint(
#     filepath='model_weight.h5', # 파일 확장자를 .keras (권장) 또는 .h5 로 지정
#     monitor='val_loss',
#     save_best_only=True,
#     save_weights_only=False,      # 전체 모델 저장 (기본값이므로 생략 가능)
#     verbose=1
# )

In [40]:
NUM_EPOCHS = 50

In [41]:
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=NUM_EPOCHS,
    verbose=1,
    callbacks=[early_stopping_cb] # 정의된 콜백 전달
    # callbacks=[early_stopping_cb, model_checkpoint_cb] # 정의된 콜백 전달
)

Epoch 1/50
969/969 [==============================] - 417s 415ms/step - loss: 1.0841 - accuracy: 0.5591 - val_loss: 0.6484 - val_accuracy: 0.7771
Epoch 2/50
969/969 [==============================] - 400s 413ms/step - loss: 0.7058 - accuracy: 0.7325 - val_loss: 0.5477 - val_accuracy: 0.7998
Epoch 3/50
969/969 [==============================] - 401s 414ms/step - loss: 0.5995 - accuracy: 0.7748 - val_loss: 0.5254 - val_accuracy: 0.8060
Epoch 4/50
969/969 [==============================] - 399s 412ms/step - loss: 0.5642 - accuracy: 0.7911 - val_loss: 0.4965 - val_accuracy: 0.8215
Epoch 5/50
969/969 [==============================] - 401s 414ms/step - loss: 0.5267 - accuracy: 0.8028 - val_loss: 0.4880 - val_accuracy: 0.8256
Epoch 6/50
969/969 [==============================] - 400s 412ms/step - loss: 0.4914 - accuracy: 0.8197 - val_loss: 0.4865 - val_accuracy: 0.8204
Epoch 7/50
969/969 [==============================] - 401s 413ms/step - loss: 0.4773 - accuracy: 0.8223 - val_loss: 0.4558 -

In [42]:
def predict(text):
    inputs = get_token_data([text], tokenizer)
    logits = model(inputs)
    return int(tf.argmax(logits, axis=1).numpy()[0])

In [43]:
test_df = pd.read_csv('test.csv', index_col=0)
submission = pd.read_csv('submission.csv', index_col=0)

In [44]:
submission['target'] = test_df['text'].apply(predict)

In [45]:
submission.to_csv('bert_klue_rm_aeda_02.csv')

In [46]:
submission

,class,target
file_name,,
t_000,NaN,1
t_001,NaN,2
t_002,NaN,0
t_003,NaN,4
t_004,NaN,4
...,...,...
t_495,NaN,2
t_496,NaN,2
t_497,NaN,1


In [47]:
model.save_weights('baseline_pretrained_rm_aeda.h5')

In [48]:
# val 예측값 실제값 비교
val_df['pred'] = val_df['conversation'].apply(predict)

In [49]:
val_df.to_csv('baseline_pretrained_rm_729_aeda_val.csv')

In [50]:
from sklearn.metrics import confusion_matrix, f1_score
cm = confusion_matrix(val_df['class'], val_df['pred'])
f1 = f1_score(val_df['class'], val_df['pred'], average='macro')

In [51]:
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[139  11   5  23   0]
 [ 15 165   3  12   0]
 [  3   3 173  14   1]
 [ 13  15   9 162   3]
 [  0   1   1   0 198]]


In [52]:
print(f1)

0.8626567956789438
